In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
studentVle_df1 = pd.read_csv('../input/ouladdata/studentVle_0.csv') 

In [ ]:
studentVle_df2 = pd.read_csv('../input/ouladdata/studentVle_1.csv') 

In [ ]:
studentVle_df3 = pd.read_csv('../input/ouladdata/studentVle_2.csv') 

In [ ]:
studentVle_df4 = pd.read_csv('../input/ouladdata/studentVle_3.csv')  

In [ ]:
studentVle_df5 = pd.read_csv('../input/ouladdata/studentVle_4.csv') 

In [ ]:
studentVle_df6 = pd.read_csv('../input/ouladdata/studentVle_5.csv') 

In [ ]:
studentVle_df7 = pd.read_csv('../input/ouladdata/studentVle_6.csv') 

In [ ]:
studentVle_df8 = pd.read_csv('../input/ouladdata/studentVle_7.csv') 

In [ ]:
studentVle_df = pd.concat([studentVle_df1,studentVle_df2, studentVle_df3, studentVle_df4, studentVle_df5, studentVle_df6, studentVle_df7, studentVle_df8])

In [ ]:
studentInfo_df = pd.read_csv('../input/ouladdata/studentInfo.csv')

In [ ]:
studentAssessment_df = pd.read_csv('../input/ouladdata/studentAssessment.csv')

In [ ]:
assessments_df = pd.read_csv('../input/ouladdata/assessments.csv')

In [ ]:
student_info = studentInfo_df
vle_activity = studentVle_df
student_assessment = studentAssessment_df
assessments = assessments_df

In [ ]:
#we only need code_module so that we can perform a join with studenAssessment and then studentInfo tables
assessments.drop(['code_presentation','assessment_type','date','weight'], axis = 1, inplace = True)
#assessments.code_module.value_counts()

In [ ]:
#merge individual assessments data with assessment data, so that we know which module each assessment belongs to
comb_assess = pd.merge(student_assessment,assessments,on='id_assessment')
comb_assess.drop(['is_banked','date_submitted'],axis = 1,inplace=True)
comb_assess.dtypes

In [ ]:
#group by student and then subgroup by module, so that we retain ability to evaluate different module predictability
grouped = comb_assess.groupby(['id_student','code_module']).mean()
grouped.sort_values('id_student')
#we can't keep id_assessment, because we have just grouped by module. We have to group by module, as our vle interaction data is 
#only grouped by module
grouped.drop(['id_assessment'],axis=1,inplace = True)
grouped.head()

In [ ]:
#put it all together
student_all_info = pd.merge(student_info,grouped,on='id_student')
#just getting a feel for it - how many modules is each student enrolled in?
fig1 = student_all_info.groupby(['id_student']).code_module.count().sort_values().hist()
fig1.set_title('Number of modules by student')
fig1.set_xlabel('Number of modules')
fig1.set_ylabel('Number of students')

In [ ]:
#total number of clicks per student by module
vle_grouped = vle_activity.groupby(['id_student','code_module']).sum()
#we have to drop the columns below as we have grouped by student and subgrouped by module, so they are meaningless
vle_grouped.drop(['id_site','date'],axis=1,inplace=True)
vle_grouped.head()

In [ ]:
#left join as we want to keep student info where no clicks were made
df = pd.merge(student_all_info,vle_grouped,on = ['id_student','code_module'],how='left')

In [ ]:
#remove rows where there are null values for sum_click. There are only 201 so it won't have a huge impact
df.dropna(inplace=True)

In [ ]:
df.code_module = pd.Categorical(df.code_module)
df.code_presentation = pd.Categorical(df.code_presentation)
df.gender = pd.Categorical(df.gender)
df.region = pd.Categorical(df.region)
df.highest_education = pd.Categorical(df.highest_education)
df.imd_band = pd.Categorical(df.imd_band)
df.age_band = pd.Categorical(df.age_band)
df.disability = pd.Categorical(df.disability)
df.final_result = pd.Categorical(df.final_result)

In [ ]:
df.isnull().sum()
df.dropna(inplace=True)


In [ ]:
df.final_result = df.final_result.replace({'Withdrawn':0,'Fail':1,'Pass':2,'Distinction':3})

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

def split_data(X, Y, pct=0.2):
    ''' 
       Divide data into training / validation set 
    '''
   # X = X.to_numpy()
   # samples = X.shape[0]
   # n_train = int(samples * pct) 
   # indices = np.arange(samples)
  #  np.random.shuffle(indices)

    # Training set
   # Xt = X[indices[:n_train]]
   # Yt = Y[indices[:n_train]]

    # Validation set
  #  Xv = X[indices[n_train:]]
   # Yv = Y[indices[n_train:]]
    
    Xt, Xv, Yt, Yv = train_test_split(X, Y, test_size=0.15)
    return (Xt, Yt), (Xv, Yv)


def fit_data(X, Y, kwargs={}, Model=DecisionTreeClassifier):
    '''
    Fit model and test performance on a validation set
    
    kwargs:
     - max_depth
     - max_leaf_nodes
     - min_impurity_split
     - min_samples_split
     - min_samples_leaf
     - criterion
     - class_weight
    '''
    training, (Xv, Yv) = split_data(X, Y)

    model = Model(**kwargs).fit(*training)
    Y_hat = model.predict(Xv)

    print('Accuracy:', accuracy_score(Yv, Y_hat))
    print('F1:', f1_score(Yv, Y_hat, average='weighted'))
    print('Confusion Matrix:\n', confusion_matrix(Yv, Y_hat))
    return model

In [ ]:
#Y = df['final_result']
#X = df.drop('final_result', axis=1)
#X = pd.get_dummies(X)

#fit_data(X, Y)

df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df.columns.values]

In [ ]:
Y = df['final_result'].replace({1:0, 2:1, 3:1}) # Fail, Withdraw, Pass, Distinction -> Incomplete, Complete
X = df.drop('final_result', axis=1)
X = pd.get_dummies(X)

fit_data(X, Y)

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer
from time import time

# Create a timing decorator 
def timer(f):
    def wrapper(*args, **kwargs):
        start  = time()
        result = f(*args, **kwargs)
        print('Execution time: %.2f seconds' % (time() - start))
        return result
    return wrapper


# Split data using sklearn
Xt, Xv, Yt, Yv = train_test_split(X, Y, test_size=0.2)

@timer
def run_search_dt(n_jobs=1):
    search = GridSearchCV(estimator = DecisionTreeClassifier(),
                          param_grid= {'max_depth'      : np.arange(2, 11),
                                       'max_leaf_nodes' : np.linspace(8, 100, 10, dtype=np.int32),
                                       'criterion'      : ['gini', 'entropy'],
                                       'class_weight'   : ['balanced', None]},
                          scoring = make_scorer(f1_score),
                          n_jobs  = n_jobs,
                          cv      = 3) # 3 fold cross validation
    search.fit(Xt, Yt)
    display(pd.DataFrame(search.cv_results_))
    print('Best score: %.3f' % search.best_score_)
    return search.best_params_
    
    
params = run_search_dt()
print(params)

In [ ]:
params = run_search_dt(4)
fit_data(X, Y, params)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from xgboost.sklearn  import XGBClassifier
#X = pd.get_dummies(X)
import re

regex = re.compile(r"\[|\]|<", re.IGNORECASE)

X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]
timed_test = timer(fit_data)

for M in [AdaBoostClassifier, XGBClassifier]:
    print(M.__name__)
    timed_test(X, Y, Model=M)
    print()

In [ ]:
model = fit_data(X, Y, Model=XGBClassifier)
importance = sorted(list(zip(X.columns, model.feature_importances_)), key=lambda k:k[1], reverse=True)
print('Top three features:')
display(importance[:3])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

imp = np.array(importance)
imp = pd.DataFrame(imp[:,1], index=imp[:,0]).T

plt.figure(figsize=(10,10));
sns.barplot(data=imp, orient='h', palette="rainbow").set_xlabel('Relative Importance %');

In [ ]:
from xgboost.plotting import plot_tree
plt.figure(figsize=(14,13));
plot_tree(model, ax=plt.gca());

In [ ]:
from sklearn.model_selection import learning_curve, ShuffleSplit

def plot_learning_curve(X, Y, Model=XGBClassifier):
    cv = ShuffleSplit(n_splits=30, test_size=0.2)
    train_sizes = np.linspace(0.1,1,8)
    train_sizes, train_scores, test_scores = learning_curve(Model(), 
                                                            X, Y, cv=cv, 
                                                            train_sizes=train_sizes, n_jobs=4)
    train_mean = np.mean(train_scores, axis=1)
    train_std  = np.std(train_scores, axis=1)
    test_mean  = np.mean(test_scores, axis=1)
    test_std   = np.std(test_scores, axis=1)

    c1 = plt.plot(train_sizes, train_mean, 'o-', label="Training score")[0].get_color()
    c2 = plt.plot(train_sizes, test_mean, 'o-', label="Cross-validation score")[0].get_color()

    plt.fill_between(train_sizes, train_mean-train_std, train_mean+train_std, alpha=0.1, color=c1)
    plt.fill_between(train_sizes, test_mean-test_std, test_mean+test_std, alpha=0.1, color=c2)

    plt.legend(loc="best")
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    
plot_learning_curve(X, Y)


In [ ]:
#vle = pd.read_csv(vle_activity)
#display(vle)

In [ ]:
studentVle_df['date'].hist();

In [ ]:
s = studentVle_df.groupby(['code_module', 'code_presentation', 'id_student', 'date'])['sum_click'].sum().reset_index()
display(s)

In [ ]:
s.groupby(['date'])['sum_click'].sum().reset_index().plot(x='date', y='sum_click', color='#bcbd22');

In [ ]:
def add_clicks(data, day):
    # Get all data points on this day or prior, and sum them over the three columns we care about
    clicks = s[s['date'] <= day].groupby(['code_module', 'code_presentation', 'id_student'])['sum_click'].sum().reset_index()
    
    # Merge this data with the appropriate rows in the full data set
    full   = pd.merge(df, clicks, on=['code_module', 'code_presentation', 'id_student'], how='outer')
    
    # Replace any missing values with 0
    full['sum_click'].fillna(0, inplace=True)
    return full

In [ ]:
from tqdm import tqdm 

# If multiprocessing isn't available, can still do the same work normally
xy_performance = []
for day in tqdm(sorted(s['date'].unique())):
    wclicks = add_clicks(df, day)
   # cleaned = clean_data(wclicks)
    
    Y = wclicks['final_result'].replace({1:0, 2:1, 3:1}) # Fail, Withdraw, Pass, Distinction -> Incomplete, Complete
    X = wclicks.drop('final_result', axis=1)
    Xt, Xv, Yt, Yv = train_test_split(X, Y, test_size=0.2)

    model = XGBClassifier().fit(Xt, Yt)
    Y_hat = model.predict(Xv)
    xy_performance.append([day, f1_score(Yv, Y_hat)])